Find the residuals between ICESat-2 elevation measurements and a reference elevation (taken from IceBridge ATM flyover of Zachariae Isstrom)

Taryn Black, ICESat-2 Hackweek, June 17-21 2019

In [40]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt

# Set data location info

In [42]:
home_dir = "/home/jovyan/xtrak/"
cross_file = "data_prod/Intersections_ATM20180418.csv"

# Load ICESat-2 crossover points

ICESat-2 track points that were identified as crossing our OIB ATM reference track in Intersections.ipynb. This file contains both the ICESat-2 elevation data and the OIB ATM data at the crossover points.

In [71]:
cross_df = pd.read_csv(home_dir + cross_file, parse_dates=[4])

cross_df.head()
#cross_df.info()

,dist_along,ATM_elev,idx_ATM,z_ATL06,t_ATL06,idx_ATL06,gt_ATL06
0,125293.549537,36.6189,3399.0,35.714657,2019-02-15 10:09:55,1578.0,gt1l
1,125388.182985,35.0684,3402.0,34.106010,2019-02-15 10:09:55,4933.0,gt1r
2,96217.510032,88.3644,2580.0,66.844490,2019-01-16 01:09:49,39471.0,gt3r
3,102715.054408,30.4860,2761.0,29.333950,2019-01-16 01:09:49,19993.0,gt1l
4,102652.754159,33.3447,2759.0,38.831944,2019-01-16 01:09:49,23915.0,gt1r


# Visual check of data

Plot reference elevation profile and crossover data points.

In [113]:
%matplotlib widget

plt.figure(figsize=(6,12)
plt.scatter(cross_df['dist_along']/1000, cross_df['ATM_elev']/1000, c='black')
plt.scatter(cross_df['dist_along']/1000, cross_df['z_ATL06']/1000, c=cross_df['t_ATL06'], s=12)
plt.xlabel('Distance along track (km)')
plt.ylabel('Elevation (m)')
plt.colorbar(label='Time')
plt.legend(['Reference track'])

SyntaxError: invalid syntax (<ipython-input-113-e305f7afe1f7>, line 4)

# Calculate residual between ICESat-2 track crossovers and reference elevation profile

For each point in the ICESat-2 crossover dataframe `df_IS`, subtract it from the reference elevation at the same distance along-track.